# python-can CAN over serial

[```python-can```](https://python-can.readthedocs.io/en/2.1.0/index.html) includes a [CAN over Serial] module.

This allows prototyping and testing with inexpensive Arduino devices before switching to another module with minimal code changes.


The packet over serial



https://python-can.readthedocs.io/en/2.1.0/interfaces/serial.html

|                | Length (Byte) | Data type               | Byte order    | Description                                   |
|----------------|---------------|-------------------------|---------------|-----------------------------------------------|
| Start of frame | 1             | Byte                    | -             | Must be 0xAA                                  |
| Timestamp      | 4             | Unsigned 4 byte integer | Little-Endian | Usually s, ms or µs since start of the device |
| DLC            | 1             | Unsigned 1 byte integer | Little-Endian | Length in byte of the payload                 |
| Arbitration ID | 4             | Unsigned 4 byte integer | Little-Endian | -                                             |
| Payload        | 0 - 8         | Byte                    | -             | -                                             |
| End of frame   | 1             | Byte                    | -             | Must be 0xBB                                  |

In [166]:
%%file SerialRead.ino
//unsigned long  __attribute__((section("VAR_BAUD"))) BAUD = 115200;
//unsigned long __attribute__((section("VAR_DELAY"))) DELAY = 1000;
//unsigned long __attribute__((section("VAR_VARBITRATION_ID"))) VARBITRATION_ID = 0x01;

unsigned long BAUD = 115200;
unsigned long DELAY = 1000;

unsigned char counter = 0;

// Arduino Setup
void setup() {
  Serial.begin(BAUD);
    // Start of frame
  Serial.write(0xAA);
// Timestamp
  Serial.write(counter);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// DLC
  Serial.write(0x01);
// Arbitration ID
  Serial.write(0x01);
  Serial.write(0x02);
  Serial.write(0x03);
  Serial.write(0x04);
// Payload
  Serial.write(counter);
// End of frame
  Serial.write(0xBB);
    
  counter++;
}

// Arduino Main Loop
void loop() {
// Start of frame
  Serial.write(0xAA);
// Timestamp
  Serial.write(counter);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// DLC
  Serial.write(0x08);
// Arbitration ID
  Serial.write(0x01);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// Payload
  Serial.write(0x01);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// End of frame
  Serial.write(0xBB);
    
  counter++;
  delay(DELAY);
}


Overwriting SerialRead.ino


In [167]:
import subprocess

stdout = subprocess.check_output(["make", "upload"]).decode()
print(stdout)

-------------------------
Arduino.mk Configuration:
- [AUTODETECTED]       CURRENT_OS = LINUX 
- [AUTODETECTED]       ARDUINO_DIR = /usr/share/arduino 
- [COMPUTED]           ARDMK_DIR = /projects/arduino_IO/arduino_make (relative to Common.mk)
- [AUTODETECTED]       ARDUINO_VERSION = 105 
- [DEFAULT]            ARCHITECTURE =  
- [DEFAULT]            ARDMK_VENDOR = arduino 
- [AUTODETECTED]       ARDUINO_PREFERENCES_PATH = /mnt/ubuntu1604_2/home/jed/.arduino/preferences.txt 
- [AUTODETECTED]       ARDUINO_SKETCHBOOK = /mnt/ubuntu1604_2/home/jed/sketchbook (from arduino preferences file)
- [BUNDLED]            AVR_TOOLS_DIR = /usr/share/arduino/hardware/tools/avr (in Arduino distribution)
- [COMPUTED]           ARDUINO_LIB_PATH = /usr/share/arduino/libraries (from ARDUINO_DIR)
- [COMPUTED]           ARDUINO_VAR_PATH = /usr/share/arduino/hardware/arduino//variants (from ARDUINO_DIR)
- [COMPUTED]           BOARDS_TXT = /usr/share/arduino/hardware/arduino//boards.txt (from ARDUINO_DIR)
- 

In [168]:
import re

In [169]:
r = re.compile("/dev/ttyUSB([\d]+)")

In [170]:
r.findall(stdout)

['2', '2', '2']

In [171]:
import serial
serial.__version__

'3.4'

In [172]:
ser_cfg=dict()
ser_cfg["port"]="/dev/ttyUSB2"
ser_cfg["baudrate"]=115200
ser_cfg["timeout"]=1
ser = serial.Serial(**ser_cfg)

In [174]:
ser.read_until(b'\xbb')

b'\xaa\x00\x00\x00\x00\x01\x01\x02\x03\x04\x00\xbb'

In [175]:
ser.read_until(b'\xbb')

b'\xaa\x01\x00\x00\x00\x08\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\xbb'

In [177]:
ser.close()

In [178]:
import can
can.__version__

'2.1.0'

In [179]:
bus = can.interface.Bus(bustype="serial",
                        channel="/dev/ttyUSB2", bitrate=115200)

In [180]:
bus.channel_info

'Serial interface: /dev/ttyUSB2'

In [181]:
packet = bus.recv()
packet

can.Message(timestamp=0.015, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0x1, dlc=8, data=[0x1, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0])

In [182]:
packet = bus.recv()
packet

can.Message(timestamp=0.016, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0x1, dlc=8, data=[0x1, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0])

In [183]:
packet.arbitration_id

1

In [184]:
packet.arbitration_id.to_bytes(4, 'little')

b'\x01\x00\x00\x00'

In [185]:
packet.arbitration_id.to_bytes(4, 'big')

b'\x00\x00\x00\x01'

In [186]:
packet.dlc

8

In [187]:
packet.timestamp.as_integer_ratio()

(1152921504606847, 72057594037927936)

In [188]:
packet.data

bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00')

In [69]:
packet.is_error_frame

False

In [71]:
packet.error_state_indicator

False

In [72]:
packet.timestamp.real

0.021

In [75]:
bus.shutdown()

In [109]:
bus.shutdown()

In [ ]:
import serial